# Mercedes Benz Greener Manufacturing
The goal of this analysis is to lower the time needed on the test bench for cars manufactured

In [2]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV
import numpy as np

In [3]:
# Load the datasets
df = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")

## Data exploration

In [47]:
# Check the first few values
df.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Check the data types and shape of the dataframe
# y is the float64 column, x0 till x8 are objects since they contain strings
# the rest are int64
df.info(), df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 378 entries, ID to X385
dtypes: float64(1), int64(369), object(8)
memory usage: 12.1+ MB


(None, (4209, 378))

In [65]:
# Check for nan values
# There are no null values
df.isnull().sum().sort_values(ascending=False)

ID      0
X254    0
X263    0
X262    0
X261    0
       ..
X127    0
X126    0
X125    0
X124    0
X385    0
Length: 378, dtype: int64

In [66]:
# Check the test set data
# As expected no target values are included
# Contains the same number of columns and rows as test
df_test

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


## Preprocessing the data

In [42]:
# Exlude the y (target)
y_train = df["y"]
df_x = df.drop("y", axis=1)
df_x

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,k,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
1,6,k,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
2,7,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
3,9,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
4,13,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,ak,s,as,c,d,aa,d,q,0,...,1,0,0,0,0,0,0,0,0,0
4205,8406,j,o,t,d,d,aa,h,h,0,...,0,1,0,0,0,0,0,0,0,0
4206,8412,ak,v,r,a,d,aa,g,e,0,...,0,0,1,0,0,0,0,0,0,0
4207,8415,al,r,e,f,d,aa,l,u,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# Check the variance of each column
df.var(numeric_only=True)

ID      5.941936e+06
y       1.607667e+02
X10     1.313092e-02
X11     0.000000e+00
X12     6.945713e-02
            ...     
X380    8.014579e-03
X382    7.546747e-03
X383    1.660732e-03
X384    4.750593e-04
X385    1.423823e-03
Length: 370, dtype: float64

In [43]:
df_x_var = df.drop(df.columns[[x for x, i in enumerate(df.var(numeric_only=True)) if i==0]], axis=1)
df_x_var

,ID,y,X0,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8405,107.39,ak,as,c,d,aa,d,q,0,...,1,0,0,0,0,0,0,0,0,0
4205,8406,108.77,j,t,d,d,aa,h,h,0,...,0,1,0,0,0,0,0,0,0,0
4206,8412,109.22,ak,r,a,d,aa,g,e,0,...,0,0,1,0,0,0,0,0,0,0
4207,8415,87.48,al,e,f,d,aa,l,u,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df.sum(axis=0)

ID                                               17702889
y                                               423717.16
X0      kkazazaztalowjhalsalonalayfnftxxyoajtfxazwxoak...
X1      vtwtvbrlsbrrbrslraacasaarbrslaasbvebssblvvslha...
X2      atavnnneeasasaqreaieasakeasmakmasamakmasasasma...
                              ...                        
X380                                                   34
X382                                                   32
X383                                                    7
X384                                                    2
X385                                                    6
Length: 378, dtype: object